In [1]:
import cv2
from PIL import Image, ImageEnhance, ImageDraw
import matplotlib.pyplot as plt

from pdf2image import convert_from_path
import pytesseract

import glob
import numpy as np
import pandas as pd
import regex as re
import warnings
warnings.filterwarnings('ignore')

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
poppler_path = r'D:\Users\chelsy.mena\Downloads\poppler-22.04.0\Library\bin'

# Cadenas de Custodia

In [28]:
archivos_remisiones = glob.glob(r'D:\Users\chelsy.mena\OneDrive - Centro de Servicios Mundial SAS\Documentos\Proyectos\HSE\Residuos\Remisiones Julio\*')

In [43]:
def delinear_pdf(archivo, lims_vert):

    # Cargar un PDF
    paginas = convert_from_path(pdf_path=archivo, poppler_path=poppler_path)

    # Convertir pagina a blanco y negro
    imagen = paginas[0].convert('L')

    # Encontrar las lineas
    ## Pasar a array de numpy
    img = np.array(imagen)

    ## Quitar fondo e invertir colores
    th, threshed = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
    threshed = cv2.morphologyEx(threshed, cv2.MORPH_OPEN, np.ones((1, 3), np.uint8)) 
    threshed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, np.ones((20, 1), np.uint8))

    # Juntar lo que es texto en un cuadrito blanco
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    dilate = cv2.dilate(threshed, kernel, iterations=4)

    ## (5) Encontrar los bordes superiores e inferiores de las lineas
    hist = cv2.reduce(dilate, 1, cv2.REDUCE_AVG).reshape(-1)

    th = 2
    H, W = img.shape[:2]
    uppers = [y for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
    lowers = [y for y in range(H-1) if hist[y]>th and hist[y+1]<=th]

    # Dibujar los Bordes en la Imagen
    img1 = ImageDraw.Draw(imagen)
    for i in range(len(uppers)):  
        img1.line([(0, uppers[i]), (1600, uppers[i])], fill ="red", width = 0)
        img1.line([(0, lowers[i]), (1600, lowers[i])], fill ="red", width = 0)
    for x in lims_vert:
        img1.line([(x, 0), (x, 2000)], fill ="red", width = 0)

    return imagen, uppers, lowers

def replicar_tabla_cadena(imagen, uppers, lowers, archivo):

    # crear Tablas para guardar ahi
    remision = pd.DataFrame({'GUIA':[],
                          'RESIDUO': [],
                          'ESTADO FÍSICO': [],
                          'EMBALAJE': [],
                          'RECOGIDO':[],
                          'UNIDADES':[],
                          'TIPOLOGÍA':[]})

    transporte = pd.DataFrame({'GUIA':[],
                          'PLACA': [],
                          'COLOR': [],
                          'TIPO': [],
                          'MARCA':[]})
    
    j = 0
    
    # Cortar por linea
    for i in range(8, len(uppers)):
        area = (0, uppers[i]-5, 1600, lowers[i]+5)
        #cropped = Image.fromarray(np.uint8(img)).crop(area)
        cropped = imagen.crop(area)
        text = pytesseract.image_to_string(cropped)

        # Parar cuando termina la tabla de residuos Y de tranporte
        if ('Empresa' in text) or ('CC' in text):
            break

        #Saltarse lo de Constancia de entrega
        if ('Observaciones' in text) or \
           ('CONSTANCIA' in text) or \
           ('Cédula' in text) or \
           ('Nombre' in text) or \
           ('Cargo' in text) or \
           ('CC' in text):
                continue

        # En que fila está lo del transporte   
        elif 'TRANSPORTE' in text:
            j = i + 1

        
        if j == i: 

            # Partir verticalmente
            limites = [0, 365, 650, 1100, 1600]

            # Sacar el No de Guia del nombre de archivo
            linea = [re.sub('\D*', '', re.escape(archivo))] 

            for k in range(1, len(limites)):
                area = (limites[k-1], 0, limites[k], cropped.size[1])
                cropped_celda = cropped.crop(area)

                text = pytesseract.image_to_string(cropped_celda)
                text = text.replace("\n", "")
                text = text.replace("Placa: ", "")
                text = text.replace("Color: ", "")
                text = text.replace("Tipo Vehiculo: ", "")
                text = text.replace("Marca: ", "")

                linea.append(text)
            
            transporte.loc[transporte.shape[0]+1] = linea

        elif j < i :
            # Partir verticalmente
            limites = [0, 392, 590, 785, 915, 1055, 1600]
            linea = [re.sub('\D*', '', re.escape(archivo))] # Sacar el No de Guia del nombre de archivo

            for k in range(1, len(limites)):
                area = (limites[k-1], 0, limites[k], cropped.size[1])
                cropped_celda = cropped.crop(area)

                text = pytesseract.image_to_string(cropped_celda)
                text = text.replace("\n", "")
                text = text.replace(" KILOGRAMOS", "")
                text = text.replace("KILOGRAMOS", "")

                linea.append(text)
            
            remision.loc[remision.shape[0]+1] = linea

    return remision, transporte

In [22]:
archivo = archivos_remisiones[0]
imagen, lims_sup, lims_inf = delinear_pdf(archivo, [0, 392, 590, 785, 915, 1055, 1600])
remisiones, transporte = replicar_tabla_cadena(imagen, lims_sup, lims_inf, archivo)

for archivo in archivos_remisiones[1:]:
    if '.pdf' in archivo:
        print("Procesando Remisión N° " + re.sub('\D*', '', re.escape(archivo)))
        imagen, lims_sup, lims_inf = delinear_pdf(archivo, [0, 392, 590, 785, 915, 1055, 1600])
        tabla1, tabla2 = replicar_tabla_cadena(imagen, lims_sup, lims_inf, archivo)
        remisiones = pd.concat([remisiones, tabla1])
        transporte = pd.concat([transporte, tabla2])

NameError: name 'archivos_remisiones' is not defined

In [31]:
transporte

,GUIA,PLACA,COLOR,TIPO,MARCA
1,574151,AAA00O,BLANCO,CAMION - ESTACAS,CHEVROLET - NHR
1,576853,AAA00O,BLANCO,CAMION - ESTACAS,CHEVROLET - NHR
1,580456,AAA00O,BLANCO,CAMION - ESTACAS,CHEVROLET - NHR
1,580457,AAA00O,BLANCO,CAMION - ESTACAS,CHEVROLET - NHR
1,581453,THL564,ROJO,CAMION - ESTACAS,FREIGHTLINER - M2 106
1,581454,THL564,ROJO,CAMION - ESTACAS,FREIGHTLINER - M2 106
1,583730,AAA00O,BLANCO,CAMION - ESTACAS,CHEVROLET - NHR


In [32]:
remisiones

,GUIA,RESIDUO,ESTADO FÍSICO,EMBALAJE,RECOGIDO,UNIDADES,TIPOLOGÍA
1,574151,LODOS DE PTAR,SOLIDO,BIG BAGS,"8.600,00",19,ORDINARIOS KG
1,576853,LODOS DE PTAR,SOLIDO,BIG BAGS,"14.600,00",22,ORDINARIOS KG
1,580456,LODOS DE PTAR,SEMISOLIDO,BIG BAGS,"14.830,00",22,ORDINARIOS KG
1,580457,LODOS DE PTAR,SEMISOLIDO,BIG BAGS,"14.370,00",18,ORDINARIOS KG
1,581453,ACEITE MINERAL,LIQUIDO,CANECA,"200,00",,DISPOSICION DIRECTA
2,581453,FIBRA DE VIDRIO - ALTO VOLUMEN,SOLIDO,CANECA,"350,00",,DISPOSICION DIRECTA -ALTO VOLUMEN
3,581453,FRASCOS Y EMPAQUESCONTAMINADOS CON PRODUCTOSQU...,SOLIDO,BOLSA,"1.380,00",,DISPOSICION DIRECTA
4,581453,RESIDUO DE FILTRACION DEEMULSIONES,SEMISOLIDO,CANECA,"2.902,00",,DISPOSICION DIRECTA
5,581453,RESIDUO IND. VARIOSCONTAMINADOS 331205,SEMISOLIDO,CANECA,"2.240,00",,ESTABILIZACION
1,581454,POLIMEROS EN SOLUCION,LIQUIDO,CANECA,"105,00",,ESTABILIZACION


# Facturas

In [24]:
archivos_facturas = glob.glob(r'D:\Users\chelsy.mena\OneDrive - Centro de Servicios Mundial SAS\Documentos\Proyectos\HSE\Residuos\facturas\*')

In [3]:
def replicar_tabla_factura(imagen, uppers, lowers, archivo):

    # crear Tabla para guardar ahi
    tabla = pd.DataFrame({'FACTURA':[],
                          'FECHA': [],
                          'GUIA':[],
                          'RESIDUO': [],
                          'PESO RECOGIDO': [],
                          'UNIDADES': [],
                          'VALOR UNITARIO': [],
                          'DESCUENTO':[],
                          'IMPUESTO':[],
                          'TOTAL':[]})
    
    # Cortar por linea
    for i in range(7, len(uppers)):
        area = (0, uppers[i]-5, 1600, lowers[i]+5)
        cropped = imagen.crop(area)
        text = pytesseract.image_to_string(cropped)

        # Parar cuando termina la tabla de residuos
        if ('CONSOLIDADO' in text) or \
           ('TOTAL' in text):
                break
        else:
            # Partir verticalmente
            limites = [0,215,320,775,900,1015,1160,1225,1390,1590]
            linea = [re.sub('\D*', '', re.escape(archivo))] # Sacar el No de factura del nombre de archivo

            for i in range(1, len(limites)):
                area = (limites[i-1], 0, limites[i], cropped.size[1])
                cropped_celda = cropped.crop(area)

                text = pytesseract.image_to_string(cropped_celda)
                text = text.replace("\n", "")
                text = text.replace("$", "")

                linea.append(text)
            
            tabla.loc[tabla.shape[0]+1] = linea

    return tabla

In [26]:
archivo = archivos_facturas[0]
imagen, lims_sup, lims_inf = delinear_pdf(archivo, [0,215,320,775,900,1015,1160,1225,1390,1590])
facturas = replicar_tabla_factura(imagen, lims_sup, lims_inf, archivo)

for archivo in archivos_facturas[1:]:
    if '.pdf' in archivo:
        print("Procesando Factura N° " + re.sub('\D*', '', re.escape(archivo)))
        imagen, lims_sup, lims_inf = delinear_pdf(archivo, [0,215,320,775,900,1015,1160,1225,1390,1590])
        tabla = replicar_tabla_factura(imagen, lims_sup, lims_inf, archivo)
        facturas = pd.concat([facturas, tabla])

facturas

Procesando Factura N° 10006
Procesando Factura N° 10140
Procesando Factura N° 10318
Procesando Factura N° 9878


,FACTURA,FECHA,GUIA,RESIDUO,PESO RECOGIDO,UNIDADES,VALOR UNITARIO,DESCUENTO,IMPUESTO,TOTAL
1,10005,24/08/2022,558857,PERSONAL,"1,00",UNIDADES,"360.000,00",,"68.400,00","428.400,00"
2,10005,24/08/2022,558857,SERVICIO MONTACARGA,"1,00",VALORPORSERVICIO,"10.080.000,00","0,00","1.915.200,00","11.995.200,00"
3,10005,24/08/2022,558857,SERVICO DE BOBCAP,"1,00",KILOGRAMOS,"2.100.000,00","0,00","399.000,00","2.499.000,00"
4,10005,24/08/2022,558857,SUCCION Y TRANSPORTE DEAGUAS Y LODOS,"1,00",HORAS,"390.000,00","0,00","74.100,00","464.100,00"
5,10005,24/08/2022,558914,RESIDUO IND. VARIOSCONTAMINADOS 331205,"990,00",KILOGRAMOS,"790,00","0,00","0,00","782.100,00"
6,10005,24/08/2022,559366,BARBOSA- LODOS DE PTARINUNDACION,"15.570,00",KILOGRAMOS,"92,00","0,00","0,00","1.432.440,00"
7,10005,25/08/2022,559369,RESIDUO IND. VARIOSCONTAMINADOS 331205,"670,00",KILOGRAMOS,"790,00","0,00","0,00","529.300,00"
1,10006,24/08/2022,558920,LODOS DE PTAR,"11.370,00",KILOGRAMOS,"92,00","0,00","0,00","1.046.040,00"
2,10006,24/08/2022,558915,RESIDUO DE FILTRACION DEEMULSIONES,"3.260,00",KILOGRAMOS,"790,00","0,00","0,00","2.575.400,00"
3,10006,24/08/2022,558915,RESIDUO IND. VARIOSCONTAMINADOS 331205,"580,00",KILOGRAMOS,"790,00","0,00","0,00","458.200,00"


# Prueba con Datos Enteros Carpeta

In [44]:
archivos_cadenas = glob.glob(r'D:\Users\chelsy.mena\Centro de Servicios Mundial SAS\ADC Barbosa - Gestión de Residuos - Gestion Residuos\Cadenas de Custodia\*.pdf')

#archivos_facturas = glob.glob(r'D:\Users\chelsy.mena\Centro de Servicios Mundial SAS\ADC Barbosa - Gestión de Residuos - Gestion Residuos\Facturas\2022\*.pdf')
archivos_facturas = glob.glob(r'D:\Users\chelsy.mena\Centro de Servicios Mundial SAS\ADC Barbosa - Gestión de Residuos - Gestion Residuos\Facturas\2023\*.pdf')
archivos_facturas = [x for x in archivos_facturas if "Anexo" in x]

In [45]:
# CADENAS DE CUSTODIA Y TRANSPORTE
archivo = archivos_cadenas[0]
imagen, lims_sup, lims_inf = delinear_pdf(archivo, [0, 392, 590, 785, 915, 1055, 1600])
cadenas, transporte = replicar_tabla_cadena(imagen, lims_sup, lims_inf, archivo)

for archivo in archivos_cadenas[1:]:
    if '.pdf' in archivo:
        print("Procesando Remisión N° " + re.sub('\D*', '', re.escape(archivo)))
        imagen, lims_sup, lims_inf = delinear_pdf(archivo, [0, 392, 590, 785, 915, 1055, 1600])
        tabla1, tabla2 = replicar_tabla_cadena(imagen, lims_sup, lims_inf, archivo)
        cadenas = pd.concat([cadenas, tabla1])
        transporte = pd.concat([transporte, tabla2])

# FACTURAS
archivo = archivos_facturas[0]
imagen, lims_sup, lims_inf = delinear_pdf(archivo, [0,215,320,775,900,1015,1160,1225,1390,1590])
facturas = replicar_tabla_factura(imagen, lims_sup, lims_inf, archivo)

for archivo in archivos_facturas[1:]:
    if '.pdf' in archivo:
        print("Procesando Factura N° " + re.sub('\D*', '', re.escape(archivo)))
        imagen, lims_sup, lims_inf = delinear_pdf(archivo, [0,215,320,775,900,1015,1160,1225,1390,1590])
        tabla = replicar_tabla_factura(imagen, lims_sup, lims_inf, archivo)
        facturas = pd.concat([facturas, tabla])

Procesando Remisión N° 616034
Procesando Remisión N° 616035
Procesando Remisión N° 616092
Procesando Remisión N° 616196
Procesando Remisión N° 618080
Procesando Remisión N° 618081
Procesando Remisión N° 618082
Procesando Remisión N° 621974
Procesando Remisión N° 621975
Procesando Remisión N° 626696
Procesando Remisión N° 626697
Procesando Remisión N° 626710
Procesando Remisión N° 626711
Procesando Remisión N° 627026
Procesando Remisión N° 627027
Procesando Factura N° 202311718
Procesando Factura N° 202311728


In [46]:
raiz = r'D:\Users\chelsy.mena\OneDrive - Centro de Servicios Mundial SAS\Documentos\Proyectos\HSE\Residuos'
facturas.to_csv(fr'{raiz}\facturas.csv')
cadenas.to_csv(fr'{raiz}\cadenas.csv')
transporte.to_csv(fr'{raiz}\transporte.csv')

In [47]:
with pd.ExcelWriter(rf'{raiz}\prueba.xlsx', engine='xlsxwriter') as writer:

    cadenas.to_excel(writer, sheet_name='Cadenas de Custodia', index=False)
    transporte.to_excel(writer, sheet_name='Datos de Transporte', index=False)
    facturas.to_excel(writer, sheet_name='Facturas', index=False)

    writer.save()

In [51]:
facturas

,FACTURA,FECHA,GUIA,RESIDUO,PESO RECOGIDO,UNIDADES,VALOR UNITARIO,DESCUENTO,IMPUESTO,TOTAL
1,202311657,22/12/2022,604349,LODOS DE PTAR,"13.790,00",KILOGRAMOS,"92,00","0,00","0,00","1.268.680,00"
2,202311657,22/12/2022,604350,LODOS DE PTAR,"13.070,00",KILOGRAMOS,"92,00","0,00","0,00","1.202.440,00"
3,202311657,28/12/2022,606654,LODOS DE PTAR,"12.510,00",KILOGRAMOS,"92,00","0,00","0,00","1.150.920,00"
4,202311657,28/12/2022,606655,LODOS DE PTAR,"13.470,00",KILOGRAMOS,"92,00","0,00","0,00","1.239.240,00"
1,202311718,20/01/2023,616033,RESIDUO DE DESTILADO DESOLVENTE PC4,"2.100,00",KILOGRAMOS,"894,00","0,00","0,00","1.877.400,00"
2,202311718,20/01/2023,616033,SALES INORGANICAS -CATALIZADOR,"3,00",KILOGRAMOS,"894,00","0,00","0,00","2.682,00"
3,202311718,20/01/202320/01/2023,616092616092,FRASCOS Y EMPAQUESCONTAMINADOS CONPRODUCTOS QU...,"750,002.085,00",KILOGRAMOSKILOGRAMOS,"894,00 894,00",,"0,00 0,00","670.500,00 1.863.990,00"
4,202311718,20/01/2023,616092,RESIDUO IND. VARIOSCONTAMINADOS 331205,"1.872,00",KILOGRAMOS,"894,00","0,00","0,00","1.673.568,00"
5,202311718,20/01/2023,616035,CHATARRA ELECTRONICA,"100,00",KILOGRAMOS,"566,00","0,00","0,00","56.600,00"
1,202311728,23/01/2023,616229,AGUAS INDUSTRAILES -BARBOSA,"36.000,00",KILOGRAMOS,"894,00","0,00","0,00","32.184.000,00"
